In [ ]:
# PREAMBLE
# system imports, set proto-models paths
import sys
import pickle
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.as_posix()))
sys.path.insert(0, str(Path.cwd().parent.parent.as_posix()))

In [ ]:
import numpy as np
from combinations import pseudo_distribution, pseudo_gen

In [ ]:
# import pathfinder to plot bam and result example
import pathfinder as pf
from pathfinder import plot_results

In [ ]:
# Fitting
from scipy.stats import norm, chi2, lognorm
from iminuit import Minuit
from iminuit.cost import UnbinnedNLL

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams["font.family"] = "serif"
plt.rcParams["axes.grid"] = False
plt.rcParams['text.kerning_factor'] = 0

In [ ]:
# Input slhafile
slhafile = '../inputFiles/slha/gluino_squarks.slha'
bd_path = 'official'

In [ ]:
# BAM Plotting
def plot_bam(bam: dict, weights: dict) -> None:
    bw = pseudo_distribution.get_bam_weight(bam, weights)
    weights = bw['weights']
    if min(weights) < 0.0:
        offset = abs(min(weights)) + 1
        weights += offset
    bam = pf.BinaryAcceptance(bw['bam'], weights=weights)
    result = pf.WHDFS(bam, ignore_subset=True)
    result.find_paths(verbose=False)
    plot_results.plot(bam, result=result, size=10)

##### Get proper result at model point defined by input file. 

In [ ]:
real_data = pseudo_gen.get_llr_at_point(slhafile, data_base=bd_path)
exp_data = pseudo_gen.get_llr_at_point(slhafile, data_base=bd_path, expected=True)

##### Plot the proper result BAM with top 10 paths

In [ ]:
plot_bam(real_data['bam'], real_data['weights'])

##### Get 500pseudo results under the SM hypothesis at model point defined by input file.

##### The results are returned as a list of weights and overlaps:

$\rightarrow$ {'weights': {'label': [weight]}, 'bam': {lebel: {set of non overlapping labels}}}

Where each weight is defined as: $w_i = -2 log_e\left[ \frac{\mathrm{L}(\mu=0|\theta)}{\mathrm{L}(\mu=1|\theta)} \right]$

In [ ]:
pseudo_data_path = Path('pseudo_data.pickle')
if pseudo_data_path.exists():
    with open(pseudo_data_path, 'rb') as handle:
        pseudo_data = pickle.load(handle)
 
else:
    pseudo_data = pseudo_gen.get_pseudo_llr(slhafile, data_base=bd_path, bootstrap_num=500, proc=4)
    with open(pseudo_data_path, 'wb') as handle:
        pickle.dump(pseudo_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

len(pseudo_data)

##### Find best sets of combinations for both the real and pseudo data 

In [ ]:
real_res = pseudo_distribution.find_best_sets([real_data], num_cor=1)
exp_res = pseudo_distribution.find_best_sets([exp_data], num_cor=1)

In [ ]:
pseudo_res_sets = pseudo_distribution.find_best_sets(pseudo_data, num_cor=2)

##### Evaluate the results

##### $x^\prime = \sum_i^N w_i $ - (N - 1)

##### $x = \frac{x^\prime - E[x^\prime]}{\sqrt{E[x^{\prime2}] -  E[x^\prime]^2}} = \frac{x^\prime - \mu_{x^\prime}}{ \sigma_{x^\prime}}$


In [ ]:
path_lengths = np.array([len(item['best']) for _, item in pseudo_res_sets.items()])
pseudo_res = np.array([item['weight'] for _, item in pseudo_res_sets.items()])


expected = (sum([exp_data['weights'][key] - 1  for key in real_res[0]['best']]) + 1)
pseudo_rescale = pseudo_res
best_rescale = real_res[0]['weight']


In [ ]:
# fid unbinned data to pdf
def data_fit(pseudo_rescale,  df=3, loc=0, scale=1) -> Minuit:
    def wrap_pdf(x, df, loc, scale):
        return chi2.pdf((x - loc)/scale, df=df)/scale
    nll = UnbinnedNLL(pseudo_rescale, wrap_pdf)
    im_fit = Minuit(nll, df, loc, scale)
    im_fit.fixed['loc'] = False
    im_fit.fixed['scale'] = False
    im_fit.fixed['df'] = True
    im_fit.scan()
    im_fit.hesse()
    return im_fit

In [ ]:
fit = data_fit(pseudo_rescale, df=len(exp_res[0]['best']), loc=expected+0.6, scale=0.7)
print(fit)
cdf = chi2.cdf(best_rescale, df=fit.values['df'], loc=fit.values['loc'], scale=fit.values['scale'])
print(f': Analytic {1. - cdf}')

##### Plot PDF

In [ ]:
fig, ax0 = plt.subplots()
xrange = (pseudo_rescale.min() - 2, pseudo_rescale.max() + 2)
xvals = np.linspace(*xrange, 1000)
ax0.hist(pseudo_rescale, bins=50, color='lightblue', edgecolor='b', histtype='step', density=True,
        label=r"$x = \frac{x\ ^\prime - \mathcal{\mu}}{\mathcal{\sigma}}$")
ax0.plot(xvals, chi2.pdf(xvals, df=fit.values['df'], loc=fit.values['loc'],  scale=fit.values['scale']), c='k', lw=0.5, ls='-')
ax0.axvline(best_rescale, ls='--', color='g', lw=2, alpha=0.2, label='Real Data point')
ax0.set_ylabel("Count", fontsize=16)
ax0.set_xlabel(r"$P(x| \theta)$", fontsize=16)
ax0.legend(loc=1, prop={'size': 16})
plt.show()

##### Plot p-value (1 - CDF) 

In [ ]:
fig, ax1 = plt.subplots()
pvals = np.array([np.sum((pseudo_rescale > val)) for val in pseudo_rescale])/(len(pseudo_rescale) - 1)
# pvals2 = norm.sf(pseudo_rescale, fit.values['mu'], fit.values['sig'])
fil_num, _, _ = ax1.hist(pvals, bins=30, color='lightblue', edgecolor='b', histtype='step', density=True, label='Histogram count')
# fil_num0, _, _ = ax1.hist(pvals2, bins=30, color='lightblue', edgecolor='r', histtype='step', density=True, label='Analytic')
# ax1.axhline(np.mean(fil_num), ls='--', color='r', lw=1, alpha=0.5)
ax1.axhline(1, ls='--', color='k', lw=1, alpha=0.5)
ax1.axvline((pseudo_rescale > best_rescale).sum()/len(pseudo_rescale-1), ls='--', color='g',
            lw=2, alpha=0.5, label='p-value, Real Data')
ax1.set_ylabel(r"$Count$", fontsize=16)
ax1.set_xlabel(r"$\mathrm{P}(X \geq x)$", fontsize=16)
ax1.legend(loc=2, prop={'size': 14})
plt.show()
plt.close()